# Data Augmentation for Medical Datasets

> This script is for medical data augmentation. \
> All algorithms are final organized version.

## Packages

In [ ]:
import os
import time
# import shap ## for XAI
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
# import pingouin as pg
import matplotlib.pyplot as plt
import sklearn.metrics as metrics

In [ ]:
from numpy import dot
from numpy.linalg import norm

from keras.models import Sequential
from keras.layers import Dense , Dropout , Lambda, Flatten
from keras.layers import Dense , Activation, Dropout, BatchNormalization
from keras.optimizers import Adam ,RMSprop
from keras.callbacks import Callback

from scipy.special import rel_entr
from scipy.stats import gaussian_kde
from scipy.spatial.distance import jensenshannon

from sklearn.model_selection import train_test_split, StratifiedKFold, KFold, train_test_split, ParameterGrid
from sklearn import decomposition, metrics
from sklearn.cluster import KMeans
from sklearn.metrics import cohen_kappa_score,f1_score, confusion_matrix, roc_auc_score, classification_report
from sklearn.inspection import permutation_importance
from sklearn.preprocessing import Normalizer, MinMaxScaler, RobustScaler
from sklearn.manifold import TSNE

from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Concatenate, Dense, Lambda, Conv1D, Flatten, Reshape, UpSampling1D, MaxPooling1D, concatenate, Layer
from tensorflow.keras.models import Model
from tensorflow.keras import layers, models, backend as K
from tensorflow.keras.losses import mse, MeanSquaredError

## Dataset

> We will apply various types/domains medical dataset. \
> It includes public dataset and even own collected medical datasets.

###  dataset check

In [ ]:
data_ori = pd.read_csv('E:/RESEARCH/Datasets/bio_data/breast_cancer_wisconsin/data.csv') ## Wisconsin breast cancer dataset

In [ ]:
print(f"The shape of the original dataset is: {data_ori.shape}")
data_ori.head()

In [ ]:
num_missing_rows = data_ori.isna().any(axis=1).sum()
print(f"The number of rows that contains at least one missing value: {num_missing_rows}")

In [ ]:
data_ori['diagnosis'].value_counts()

### Unnecessary feature elimination

> Unnecessary features varies according to the analysis purpose. \
> Researcher should provide research goal or purpose of the augmentation model.

In [ ]:
drop_list = ['Unnamed: 32','id']
data_ori_ = data_ori.drop(drop_list, axis = 1 )

### Scaler on x values

In [ ]:


x_only[:] = (scaler.fit_transform(x_only[:])).round(decimals=6)

***

***

# Data Augmentations

## dataset for model training

In [ ]:
data = data_ori_.copy()

In [ ]:
data_B = data[data['diagnosis']=="B"]
data_M = data[data['diagnosis']=="M"]

In [ ]:
data_vae = data.copy()
# data_vae = data_B.copy()
# data_vae = data_M.copy()

## Variational Autoencoder (VAE)

### preparing x, y dataset

> Applying scaler on dataset prevents "nan loss" issue during VAE training

In [ ]:
x = data_vae.drop("diagnosis", axis=1)
x = x.fillna(x.mean()) ## filling na values with mean values (just drop the rows is also a possible option)
y = data_vae.diagnosis

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

data_x = x.copy()
data_x[:] = (scaler.fit_transform(data_x[:])).round(decimals=6) ## scaling x values
label = y.copy()
label = label.replace({'B':0})
label = label.replace({'M': 1})
data_y = to_categorical(label, 2) ## into the format of one-hot encoding

In [ ]:
print("The size of x dataset is:", data_x.shape)
print("The size of y dataset is:", data_y.shape)

### VAE model

In [ ]:
class VAE:
    def __init__(self, input_dim, latent_dim):
        self.input_dim = input_dim
        self.latent_dim = latent_dim
        self.encoder = None
        self.decoder = None
        self.vae = None
        self.build_model()
        self.callbacks = []

    ## VAE model for data augmentation
    def build_model(self):
        ####################################################
        ## defining encoder section
        inputs = Input(shape=(self.input_dim,))
        
        # shallow model use
        h = Dense(16, activation='relu')(inputs)

        # # deeper model use
        # h = Dense(32, activation='relu')(inputs)
        # h = Dense(16, activation='relu')(h)
        # h = Dense(8, activation='relu')(h)

        # calculating mean/var
        z_mean = Dense(self.latent_dim)(h)
        z_log_var = Dense(self.latent_dim)(h)

        ## sampling section (for gaussian distribution)
        def sampling(args):
            z_mean, z_log_var = args
            batch = K.shape(z_mean)[0]
            dim = K.int_shape(z_mean)[1]
            epsilon = K.random_normal(shape=(batch, dim))
            return z_mean + K.exp(0.5 * z_log_var) * epsilon

        z = Lambda(sampling, output_shape=(self.latent_dim,))([z_mean, z_log_var])

        ####################################################
        ## defining decoder section

        # shallower model use
        decoder_h = Dense(16, activation='relu')
        decoder_mean = Dense(self.input_dim, activation='sigmoid')
        h_decoded = decoder_h(z)
        x_decoded_mean = decoder_mean(h_decoded)
        
        # # deeper model use
        # decoder_h1 = Dense(8, activation='relu')
        # decoder_h2 = Dense(16, activation='relu')
        # decoder_h3 = Dense(32, activation='relu')
        # decoder_mean = Dense(self.input_dim, activation='sigmoid')

        # h_decoded = decoder_h1(z)
        # h_decoded = decoder_h2(h_decoded)
        # h_decoded = decoder_h3(h_decoded)
        # x_decoded_mean = decoder_mean(h_decoded)

        ####################################################
        ## defining VAE model
        self.vae = Model(inputs, x_decoded_mean)

        ####################################################
        ## defining loss function (reconstruction + KL divergence)
        reconstruction_loss = MeanSquaredError()(inputs, x_decoded_mean)
        kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var + K.epsilon()), axis=-1)
        vae_loss = K.mean(reconstruction_loss + kl_loss)
        self.vae.add_loss(vae_loss)

        ####################################################
        ## compiling model
        self.vae.compile(optimizer=Adam(learning_rate=0.001))

        ####################################################
        ## Extracting encoder and decoder separately
        ## encoder section
        self.encoder = Model(inputs, z_mean)

        ## decoder section
        #### shallow model
        decoder_input = Input(shape=(self.latent_dim,))
        _h_decoded = decoder_h(decoder_input)
        _x_decoded_mean = decoder_mean(_h_decoded)
        self.decoder = Model(decoder_input, _x_decoded_mean)
        
        # #### deeper model
        # decoder_input = Input(shape=(self.latent_dim,))
        # _h_decoded = decoder_h1(decoder_input)
        # _h_decoded = decoder_h2(_h_decoded)
        # _h_decoded = decoder_h3(_h_decoded)
        # _x_decoded_mean = decoder_mean(_h_decoded)
        # self.decoder = Model(decoder_input, _x_decoded_mean)
    
    ####################################################
    ####################################################
    ## Model training
    def train(self, data, epochs, batch_size, validation_split):
        self.vae.fit(data, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

    def encode(self, data):
        return self.encoder.predict(data)

    def decode(self, latent_points):
        return self.decoder.predict(latent_points)

    def generate_synthetic_data(self, num_samples=1):
        latent_samples = np.random.normal(size=(num_samples, self.latent_dim)) ## sampling from latent space
        return self.decode(latent_samples) ## synthetic data generation with decoder section

    def visualize_latent_space(self, data, labels):
        ## encode the dataset into latent space
        encoded_data = self.encode(data)
        ## latent space visualization with t-SNE
        tsne = TSNE(n_components=2, random_state=710674)
        encoded_data_tsne = tsne.fit_transform(encoded_data)
        ## Visualize
        plt.figure(figsize=(8, 6))
        plt.scatter(encoded_data_tsne[:, 0], encoded_data_tsne[:, 1], c=labels, cmap='viridis')
        plt.colorbar()
        plt.xlabel("t-SNE component 1")
        plt.ylabel("t-SNE component 2")
        plt.title("t-SNE visualization of the latent space")
        plt.show()

In [ ]:
vae = VAE(input_dim=data_x.shape[1], latent_dim=4)
vae.train(data_x, epochs = 150, batch_size = 32, validation_split=0.2)

#### Latent space visualization

In [ ]:
vae.visualize_latent_space(data_x, label)

#### Synthetic data generation

* For mdd class augmentation

In [ ]:
synthetic_mdd = vae.generate_synthetic_data(num_samples=10)
gen_mdd = synthetic_mdd.copy()
gen_mdd = pd.DataFrame(gen_mdd, columns = data_x.columns)

* For pd class augmentation

In [ ]:
synthetic_pd = vae.generate_synthetic_data(num_samples=10)
gen_pd = synthetic_pd.copy()
gen_pd = pd.DataFrame(gen_pd, columns = data_x.columns)

* For con class augmentation

In [ ]:
synthetic_con = vae.generate_synthetic_data(num_samples=10)
gen_con = synthetic_con.copy()
gen_con = pd.DataFrame(gen_con, columns = data_x.columns)

***

## Conditional Variational Autoencoder (CVAE)

> Advantage of CVAE is adding label or more information to increase explainability.

### preparing x, y dataset

> Applying scaler on dataset prevents "nan loss" issue during VAE training

In [ ]:
data_vae.columns

In [ ]:
x = data_vae.drop("diagnosis", axis=1)
x = x.fillna(x.mean()) ## filling na values with mean values (just drop the rows is also a possible option)
y = data_vae.diagnosis
# c = data_vae.loc[:,["diagnosis"]]
c = data_vae.iloc[:,0]

In [ ]:
scaler = MinMaxScaler() #set the scaler (between 0 and 1)

data_x = x.copy()
data_x[:] = (scaler.fit_transform(data_x[:])).round(decimals=6) ## scaling x values
label = y.copy()
label = label.replace({'B':0})
label = label.replace({'M': 1})
data_y = to_categorical(label, 2) ## into the format of one-hot encoding
data_c = c.copy()

In [ ]:
print("The size of x dataset is:", data_x.shape)
print("The size of y dataset is:", data_y.shape)
print("The size of c dataset is:", data_c.shape)

### CVAE model

In [ ]:
class CVAE:
    def __init__(self, input_dim, condition_dim, latent_dim):
        ## initializing data demensionality
        self.input_dim = input_dim
        self.condition_dim = condition_dim
        self.latent_dim = latent_dim
        
        ## generating encoder and decoder section
        self.encoder = self.build_encoder()
        self.decoder = self.build_decoder()
        self.cvae = self.build_cvae()
    
    ## defining encoder function
    def build_encoder(self):
        x_input = layers.Input(shape=(self.input_dim,), name='input')
        c_input = layers.Input(shape=(self.condition_dim,), name='condition')
        
        combined_input = layers.Concatenate()([x_input, c_input])
        
        h = layers.Dense(16, activation='relu')(combined_input)
        # h = layers.Dense(32, activation='relu')(h)
        
        z_mean = layers.Dense(self.latent_dim, name='z_mean')(h)
        z_log_var = layers.Dense(self.latent_dim, name='z_log_var')(h)
        
        ## sampling (reparameterization)
        def sampling(args):
            z_mean, z_log_var = args
            batch = K.shape(z_mean)[0]
            dim = K.int_shape(z_mean)[1]
            epsilon = K.random_normal(shape=(batch, dim))
            return z_mean + K.exp(0.5 * z_log_var) * epsilon
        
        z = layers.Lambda(sampling, output_shape=(self.latent_dim,), name='z')([z_mean, z_log_var])
        
        return Model([x_input, c_input], [z_mean, z_log_var, z], name='encoder')
    
    ## defining decoder function
    def build_decoder(self):
        z_input = layers.Input(shape=(self.latent_dim,), name='z_sampling')
        c_input = layers.Input(shape=(self.condition_dim,), name='condition')
        
        combined_input = layers.Concatenate()([z_input, c_input])
        
        h = layers.Dense(16, activation='relu')(combined_input)
        # h = layers.Dense(64, activation='relu')(h)
        x_decoded = layers.Dense(self.input_dim, activation='sigmoid', name='output')(h)
        
        return Model([z_input, c_input], x_decoded, name='decoder')
    
    ## build and compile the CVAE model
    def build_cvae(self):
        x_input = layers.Input(shape=(self.input_dim,), name='input')
        c_input = layers.Input(shape=(self.condition_dim,), name='condition')
        
        z_mean, z_log_var, z = self.encoder([x_input, c_input])
        x_decoded = self.decoder([z, c_input])
        
        cvae = Model([x_input, c_input], x_decoded, name='cvae')
        
        reconstruction_loss = mse(x_input, x_decoded)
        reconstruction_loss *= self.input_dim
        
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        
        cvae_loss = K.mean(reconstruction_loss + kl_loss)
        cvae.add_loss(cvae_loss)
        cvae.compile(optimizer='adam')
        
        return cvae
    
    ## model training
    def train(self, x_data, c_data, epochs, batch_size, validation_split):
        self.cvae.fit([x_data, c_data], epochs=epochs, batch_size=batch_size, validation_split = validation_split)
    
    ## synthetic data generation (for data augmentation)
    def generate_synthetic_data(self, condition, n_samples=1):
        z_samples = np.random.normal(size=(n_samples, self.latent_dim))
        synthetic_data = self.decoder.predict([z_samples, condition])
        return synthetic_data
        
    def visualize_latent_space(self, x_data, c_data, labels, n_samples):
        n_samples = min(n_samples, len(x_data), len(c_data))
        indices = np.random.choice(len(x_data), n_samples, replace=False) ## data sampling for n
        x_sample = x_data.iloc[indices]
        c_sample = c_data[indices]
        z_mean, _, _ = self.encoder.predict([x_sample, c_sample]) ## calculating latent vector z
        tsne = TSNE(n_components=2, random_state=710674) #tsne visualization
        z_tsne = tsne.fit_transform(z_mean)

        ## visualization
        plt.figure(figsize=(8, 6))
        if labels is not None:
            plt.scatter(z_tsne[:, 0], z_tsne[:, 1], c=labels[indices], cmap='viridis')
            plt.colorbar()
        else:
            plt.scatter(z_tsne[:, 0], z_tsne[:, 1])
        plt.title('t-SNE visualization of the latent space')
        plt.xlabel('t-SNE 1')
        plt.ylabel('t-SNE 2')
        plt.show()

In [ ]:
cvae = CVAE(input_dim = data_x.shape[1], condition_dim = data_y.shape[1], latent_dim=2)
cvae.train(data_x, data_y, epochs=150, batch_size=32, validation_split=0.2)

#### Latent space visualization

In [ ]:
cvae.visualize_latent_space(data_x, data_y, label, n_samples=300)

***

***

# Classification performance check

## Original dataset only

In [ ]:
data_dnn = data_ori_.copy()

In [ ]:
x = data_dnn.drop("diagnosis", axis=1)
x = x.fillna(x.mean()) ## filling na values with mean values (just drop the rows is also a possible option)
y = data_dnn.diagnosis

In [ ]:
data_x = x.copy()
data_x[:] = (scaler.fit_transform(data_x[:])).round(decimals=6) ## scaling x values

label = y.copy()
label = label.replace({'B':0})
label = label.replace({'M': 1})
data_y = to_categorical(label, 2) ## into the format of one-hot encoding

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(data_x, data_y, test_size = 0.2, random_state = 710674)

In [ ]:
x_train.shape

In [ ]:
class_weight = {0:1, 1:1}

## Augmented dataset add

## Simple DNN model

> To compare classification performance, we design simple structure of DNN.

### DNN model

In [ ]:
class SimpleDNN:
    def __init__(self, input_dim, layer_configs, output_units=2, output_activation='softmax'):
        self.input_dim = input_dim ## input data dimensionality (# variables)
        self.layer_configs = layer_configs ## hidden layer/sequential layer lists (units, activation, batch_norm, dropout_rate)
        self.output_units = output_units ## output unit no.
        self.output_activation = output_activation ## activation function for output layer
        self.model = self.build_model()
        self.callbacks = []

    def build_model(self):
        model = models.Sequential()
        ## add first hidden layer
        model.add(layers.Dense(units=self.layer_configs[0]['units'], activation=self.layer_configs[0]['activation'], input_shape=(self.input_dim,)))
        
        ## batch normalization and dropout for first layer
        if self.layer_configs[0].get('batch_norm', False):
            model.add(layers.BatchNormalization())
        if self.layer_configs[0].get('dropout_rate', None) is not None:
            model.add(layers.Dropout(rate=self.layer_configs[0]['dropout_rate']))
        
        ## do same for rest hidden layers (except for the last)
        for config in self.layer_configs[1:]:
            model.add(layers.Dense(units=config['units'], activation=config['activation']))
            
            if config.get('batch_norm', False):
                model.add(layers.BatchNormalization())
            
            if config.get('dropout_rate', None) is not None:
                model.add(layers.Dropout(rate=config['dropout_rate']))
        
        ## add output layer
        model.add(layers.Dense(units=self.output_units, activation=self.output_activation))
        
        return model

    def compile_model(self, optimizer, loss='categorical_crossentropy', metrics=['accuracy'], lr_scheduler=None):
        if lr_scheduler:
            ## AddLearningRateScheduler callback
            self.callbacks.append(LearningRateScheduler(lr_scheduler))
        
        self.model.compile(optimizer, loss, metrics)

    def fit_model(self, x_train, y_train, epochs, batch_size, validation_split):
        return self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split,
                             callbacks=self.callbacks)

    def evaluate_model(self, x_test, y_test):
        return self.model.evaluate(x_test, y_test)

    def predict(self, x):
        return self.model.predict(x)

    def summary(self):
        self.model.summary()

In [ ]:
def dynamic_learning_rate(epoch, mode='cyclic', base_lr=0.001, max_lr=0.006, step_size=2000, gamma=0.99994):
    cycle = np.floor(1 + epoch / (2 * step_size))
    x = np.abs(epoch / step_size - 2 * cycle + 1)
    
    if mode == 'cyclic' or mode == 'triangular':
        lr = base_lr + (max_lr - base_lr) * max(0, (1 - x))
    elif mode == 'triangular2':
        lr = base_lr + (max_lr - base_lr) * max(0, (1 - x)) / (2 ** (cycle - 1))
    elif mode == 'exp_range':
        lr = base_lr + (max_lr - base_lr) * max(0, (1 - x)) * (gamma ** epoch)
    else:
        raise ValueError("Invalid mode. Choose from 'cyclic', 'triangular', 'triangular2', or 'exp_range'.")
    
    return lr

In [ ]:
# lr = dynamic_learning_rate(epoch=1000, mode='cyclic')
# lr = dynamic_learning_rate(epoch=1000, mode='triangular')
# lr = dynamic_learning_rate(epoch=1000, mode='triangular2')
# lr = dynamic_learning_rate(epoch=1000, mode='exp_range', gamma=0.99994)

In [ ]:
## model initialization with hidden layer list below
layer_configs = [
    {'units': 32, 'activation': 'relu', 'batch_norm': True, 'dropout_rate': 0.5},
    {'units': 16, 'activation': 'relu', 'batch_norm': False, 'dropout_rate': 0.3},
    {'units': 8, 'activation': 'relu', 'batch_norm': True, 'dropout_rate': 0.2}
]

model = SimpleDNN(input_dim=x_train.shape[1], layer_configs=layer_configs)

In [ ]:
## compile our model
opt = keras.optimizers.SGD(learning_rate = 0.001, decay = 1e-5, momentum = 0.9)
scheduler = lambda epoch: dynamic_learning_rate(epoch, mode='triangular2', base_lr=0.001, max_lr=0.009, step_size=25)
model.compile_model(optimizer = opt, lr_scheduler=scheduler)

## model summary
model.summary()



# 모델 평가
# loss, accuracy = mlp.evaluate_model(x_test, y_test)

# 예측
# predictions = model.predict(x_new)

In [ ]:
## model training on training dataset
history = model.fit_model(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

### Evaluation

In [ ]:
def evaluate_model_performance(model, x_test, y_test):

    ## predict on model
    y_predict = model.predict(x_test)
    y_predict_classes = np.argmax(y_predict, axis=1)
    y_test_classes = np.argmax(y_test, axis=1)
    
    ## calculate confusion matrix and visualize
    cm = confusion_matrix(y_test_classes, y_predict_classes, normalize='pred')
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, cmap=plt.cm.Blues, fmt='.2f')
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.title('Confusion Matrix')
    plt.show()
    
    ## evaluation metrics
    accuracy = metrics.accuracy_score(y_test_classes, y_predict_classes)
    precision = metrics.precision_score(y_test_classes, y_predict_classes, average='macro')
    recall = metrics.recall_score(y_test_classes, y_predict_classes, average='micro')
    f1 = metrics.f1_score(y_test_classes, y_predict_classes, average='weighted')
    auc = roc_auc_score(y_test, y_predict, multi_class='ovr')
    
    ## Results
    print("=============================================")
    print(f"The overall accuracy is: {accuracy:.4f}")
    print(f"The precision score is: {precision:.4f}")
    print(f"The recall score is: {recall:.4f}")
    print(f"The F1 score is: {f1:.4f}")
    print(f"The AUC score is: {auc:.4f}")
    print("=============================================")
    
    ## Print out the classification report
    print("\nClassification Report:")
    print(classification_report(y_test_classes, y_predict_classes))

In [ ]:
evaluate_model_performance(model, x_test, y_test)